In [35]:
# Initial imports
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import datetime
import numpy as np
#import tensorflow as tf

#  Import and read the file.
supply_df = pd.read_csv("C:/Users/Elissa/Desktop/CAISO-ALL-SUPPLY-DATA.csv")
supply_df.head()


,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Unix Timestamp
0,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,4/10/2018,00:00:00,2018-04-10 00:00:00,1.523318e+09
1,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,4/10/2018,00:05:00,2018-04-10 00:05:00,1.523319e+09
2,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,4/10/2018,00:10:00,2018-04-10 00:10:00,1.523319e+09
3,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,4/10/2018,00:15:00,2018-04-10 00:15:00,1.523319e+09
4,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,4/10/2018,00:20:00,2018-04-10 00:20:00,1.523320e+09


In [36]:
supply_df['Sum'] = supply_df['Renewables'] + supply_df['Large hydro'] + supply_df['Imports'] + supply_df['Batteries'] + supply_df['Nuclear'] + supply_df['Coal'] + supply_df['Other'] + supply_df['Natural gas']


In [37]:
supply_df.dtypes

Renewables        float64
Natural gas       float64
Large hydro       float64
Imports           float64
Batteries         float64
Nuclear           float64
Coal              float64
Other             float64
Date               object
Time               object
DateTime           object
Unix Timestamp    float64
Sum               float64
dtype: object

In [38]:
baby_df = supply_df['Time'].str.split(":", expand=True)
print(baby_df)

         0   1   2
0       00  00  00
1       00  05  00
2       00  10  00
3       00  15  00
4       00  20  00
...     ..  ..  ..
412951  23  35  00
412952  23  40  00
412953  23  45  00
412954  23  50  00
412955  23  55  00

[412956 rows x 3 columns]


In [39]:
baby_df[0] = baby_df[0].astype(int)
baby_df[1] = baby_df[1].astype(int)

In [40]:
baby_df= baby_df.drop([2],1)

In [41]:
baby_df.dtypes

0    int32
1    int32
dtype: object

In [42]:
# Rename columns and create years value for calculating demand_EV
baby_df = baby_df.rename(columns={baby_df.columns[0]: 'Hours'})
baby_df = baby_df.rename(columns={baby_df.columns[1]: 'Minutes'})

In [43]:
supply_df['Date'] = pd.to_datetime(supply_df['Date'])

#Create a new column for'Year'
baby_df['Year'] = supply_df['Date'].dt.year

#Convert to integers to be used in computations
baby_df['Year'] = baby_df['Year'].astype(int)



baby_df.head(100)

,Hours,Minutes,Year
0,0,0,2018
1,0,5,2018
2,0,10,2018
3,0,15,2018
4,0,20,2018
...,...,...,...
95,7,55,2018
96,8,0,2018
97,8,5,2018
98,8,10,2018


In [44]:
baby_df['DemandMW'] = 0.0

mask = (baby_df['Year'] == 2018) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask1 = (baby_df['Year'] == 2019) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask2 = (baby_df['Year'] == 2020) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)


baby_df.loc[mask,'DemandMW'] = 6.2356
baby_df.loc[mask1,'DemandMW'] = 8.2818
baby_df.loc[mask2,'DemandMW'] = 9.9174

In [45]:
baby_df

,Hours,Minutes,Year,DemandMW
0,0,0,2018,6.2356
1,0,5,2018,6.2356
2,0,10,2018,6.2356
3,0,15,2018,6.2356
4,0,20,2018,6.2356
...,...,...,...,...
412951,23,35,2022,9.9174
412952,23,40,2022,9.9174
412953,23,45,2022,9.9174
412954,23,50,2022,9.9174


In [46]:
supply_df['Date'] = pd.to_datetime(supply_df['Date'])

In [47]:
def fun(datetimething):
     return datetimething.timetuple().tm_yday

In [48]:
supply_df['Date'] = supply_df['Date'].apply(fun)
supply_df.head(1300)

,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Unix Timestamp,Sum
0,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,100,00:00:00,2018-04-10 00:00:00,1.523318e+09,22879.0
1,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,100,00:05:00,2018-04-10 00:05:00,1.523319e+09,22794.0
2,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,100,00:10:00,2018-04-10 00:10:00,1.523319e+09,22743.0
3,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,100,00:15:00,2018-04-10 00:15:00,1.523319e+09,22488.0
4,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,100,00:20:00,2018-04-10 00:20:00,1.523320e+09,22368.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,3262.0,4010.0,3096.0,5808.0,-3.0,2253.0,7.0,0.0,105,04:15:00,2018-04-15 04:15:00,1.523766e+09,18433.0
1296,3206.0,4027.0,3102.0,5786.0,0.0,2253.0,7.0,0.0,105,04:20:00,2018-04-15 04:20:00,1.523766e+09,18381.0
1297,3187.0,4076.0,3106.0,5783.0,0.0,2252.0,7.0,0.0,105,04:25:00,2018-04-15 04:25:00,1.523766e+09,18411.0
1298,3144.0,4018.0,3103.0,5879.0,0.0,2253.0,7.0,0.0,105,04:30:00,2018-04-15 04:30:00,1.523767e+09,18404.0


In [49]:
final_df = pd.merge(baby_df, supply_df, left_index=True, right_index=True, how='outer')

In [50]:
final_df.head()

,Hours,Minutes,Year,DemandMW,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Unix Timestamp,Sum
0,0,0,2018,6.2356,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,100,00:00:00,2018-04-10 00:00:00,1.523318e+09,22879.0
1,0,5,2018,6.2356,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,100,00:05:00,2018-04-10 00:05:00,1.523319e+09,22794.0
2,0,10,2018,6.2356,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,100,00:10:00,2018-04-10 00:10:00,1.523319e+09,22743.0
3,0,15,2018,6.2356,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,100,00:15:00,2018-04-10 00:15:00,1.523319e+09,22488.0
4,0,20,2018,6.2356,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,100,00:20:00,2018-04-10 00:20:00,1.523320e+09,22368.0


In [ ]:
#outliers_df = supply_df['Natural gas'][supply_df['Natural gas'] < 5000]
#outliers_df.head()